<a href="https://colab.research.google.com/github/marquezo/darknet/blob/master/train_yolo_duckie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training an Object Detector for Duckietown
By orlando.m.bol@gmail.com

YOLO is a fast object detector with decent accuracy. It is real-time and thus useful for the Duckietown environment. For more information, see https://pjreddie.com/darknet/yolo/.

## Clone a forked version of the YOLO3 repo that contains the Duckietown datasets

In [ ]:
!git clone https://github.com/marquezo/darknet.git

fatal: destination path 'darknet' already exists and is not an empty directory.


## Install CUDA and compile Darknet (which YOLO is built on)

In [ ]:
!wget -O cuda_9.2.88_396.26_linux.run -c https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux

--2020-09-10 07:00:26--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?dYtiL6qW1LC3FjPDQbIkCLKn_S4aB2lPlvBWHee4qwOVGQlW0GLThEjs3ekvXLUF8LZydDi82MNvjT3EZJOSw5fcp5v6awRRFYjQlU8hPQhdLfiCoMMkHK3_WWQJPTyRqmf7u3NGXx7WmmeQEIx9FDVsIWvf-Im2yLGWf026Z6O8D4fYQH2g24g [following]
--2020-09-10 07:00:27--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?dYtiL6qW1LC3FjPDQbIkCLKn_S4aB2lPlvBWHee4qwOVGQlW0GLThEjs3ekvXLUF8LZydDi82MNvjT3EZJOSw5fcp5v6awRRFYjQlU8hPQhdLfiCoMMkHK3_WWQJPTyRqmf7u3NGXx7WmmeQEIx9FDVsIWvf-Im2yLGWf026Z6O8D4fYQH2g24g
Resolving developer.download.

In [ ]:
!chmod +x cuda_9.2.88_396.26_linux.run
!./cuda_9.2.88_396.26_linux.run --verbose --silent --toolkit --override

Installing the CUDA Toolkit in /usr/local/cuda-9.2 ...
Verifying archive integrity... All good.

Uncompressing NVIDIA CUDA..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Include cuda in the PATH and compile Darknet**

Note that the Makefile has been modified to compile with CUDA and without OPENCV

In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda-9.2/bin'

In [ ]:
cd darknet

/content/darknet


In [ ]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -W

## Test YOLO installation

In [ ]:
# Download pre-trained weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2020-09-10 07:09:08--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   193KB/s    in 16m 17s 

2020-09-10 07:25:27 (248 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
# Run object detection on sample image
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

## Train tiny YOLO on Duckietown images
We realized that the more shallow version of YOLO, named tiny YOLO, is good enough for Duckietown. Since it has fewer convolutional layers, it trains and predicts faster. You can stop the training when you like and use the saved weights to do inference.

In [ ]:
# Download pre-trained weights to effectively do transfer learning
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-09-10 07:27:37--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   117KB/s    in 13m 5s  

2020-09-10 07:40:43 (202 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
# Create a directory where we will save backups of the weights
!mkdir duckie_backup

In [ ]:
# Train tiny YOLO using the already created configuration files
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector train cfg/duckie-multi.data cfg/yolov3-tiny-duckie-multi.cfg darknet53.conv.74

yolov3-tiny-duckie-multi
layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->   

## Test your Duckietown Object Detector
Pick any image from the directory `duckiestuff/testset`

In [ ]:
# Let's see what has been saved during training
!ls -hlt duckie_backup/

total 332M
-rw-r--r-- 1 root root 34M Sep 10 08:40 yolov3-tiny-duckie-multi.backup
-rw-r--r-- 1 root root 34M Sep 10 08:22 yolov3-tiny-duckie-multi_900.weights
-rw-r--r-- 1 root root 34M Sep 10 08:18 yolov3-tiny-duckie-multi_800.weights
-rw-r--r-- 1 root root 34M Sep 10 08:13 yolov3-tiny-duckie-multi_700.weights
-rw-r--r-- 1 root root 34M Sep 10 08:09 yolov3-tiny-duckie-multi_600.weights
-rw-r--r-- 1 root root 34M Sep 10 08:05 yolov3-tiny-duckie-multi_500.weights
-rw-r--r-- 1 root root 34M Sep 10 08:01 yolov3-tiny-duckie-multi_400.weights
-rw-r--r-- 1 root root 34M Sep 10 07:56 yolov3-tiny-duckie-multi_300.weights
-rw-r--r-- 1 root root 34M Sep 10 07:51 yolov3-tiny-duckie-multi_200.weights
-rw-r--r-- 1 root root 34M Sep 10 07:47 yolov3-tiny-duckie-multi_100.weights


In [26]:
# Run inference
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector test cfg/duckie-multi.data cfg/yolov3-tiny-duckie-multi.cfg duckie_backup/yolov3-tiny-duckie-multi.backup data_folder/testset/frame0012.jpg -thresh .50

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

## Transferring weights to Google Drive
You will want to transfer the trained weights to some other computer so that you can run inference. The easiest way is to copy them to your Google drive and then download the weights from there.

In [ ]:
# Map your Google drive to Collab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGmHYKDM11COjCPe_6EN8Rg1exZqVyzynu0-sZo3ldHvCz15Jsi8R0
Mounted at /content/gdrive


In [27]:
# Copy weights to google drive
!cp duckie_backup/yolov3-tiny-duckie-multi.backup /content/gdrive/My\ Drive/tiny-yolo-duckie.weights